In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tarfile
import os
import fnmatch
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import pandas as pd
import numpy as np
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision
from torchvision import models
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
fname = './drive/MyDrive/line.tar'
ap = tarfile.open(fname)
ap.extractall('./line')
ap.close()
data_dir = './line/line'
file_list = os.listdir(data_dir)
image_paths = []
states = []
pattern = "*.png"
for filename in file_list:
  if fnmatch.fnmatch(filename, pattern):
    image_paths.append(os.path.join(data_dir,filename))
    state = str(filename[-5:-4])
    states.append(state)
images = [cv2.imread(img) for img in image_paths]
L_data =[]
R_data = []
G_data = []
for i in range(len(states)):
  if states[i] == "L":
    L_data.append(images[i])
  elif states[i] == "R":
    R_data.append(images[i])
  elif states[i] == "G":
    G_data.append(images[i])
images = []
states = []
for i in range(len(L_data)):
  images.append(L_data[i])
  states.append("L")
for i in range(len(R_data)):
  images.append(R_data[i])
  states.append("R")
for i in range(len(G_data)):
  images.append(G_data[i])
  states.append("G")
df = pd.DataFrame()
df['state'] = states
df['state'].value_counts()
from sklearn.preprocessing import LabelEncoder
# OneHotEncoder 생성 및 학습 데이터에 fit
onehot_encoder = OneHotEncoder(sparse=False)
labels_encoded = onehot_encoder.fit_transform(np.array(['G', 'L', 'R']).reshape(-1, 1))
transform = transforms.Compose([
                                #transforms.RandomHorizontalFlip(),
                                #transforms.RandomCrop((320, 640), padding=4),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                ])
class CustomDataset(Dataset):
    def __init__(self, images, states, transform=None, train=True, onehot_encoder=None):
        self.images = images
        self.states = states
        self.transform = transform
        self.train = train
        self.onehot_encoder = onehot_encoder
        # 데이터셋을 train과 test로 나누기
        train_size = int(0.8 * len(self.images))
        test_size = len(self.images) - train_size
        train_data, test_data = train_test_split(list(zip(self.images, self.states)), test_size=test_size, random_state=42)
        if self.train:
            self.data = train_data
        else:
            self.data = test_data
    def __len__(self):
        return len(self.data)
    # def __getitem__(self, idx):
    #     img_path, state = self.data[idx]
    #     img = Image.fromarray(np.asarray(img_path)).convert('RGB')
    #     if self.transform:
    #         img = self.transform(img)
    #     # state를 숫자로 라벨링
    #     state = self.label_encoder.transform([state])[0]
    #     return img, state
    def __getitem__(self, idx):
        img_path, state = self.data[idx]
        img = Image.fromarray(np.asarray(img_path)).convert('RGB')
        if self.transform:
            img = self.transform(img)
        # state를 숫자로 라벨링
        # state = self.label_encoder.transform([state])[0]
        # state = torch.tensor(state, dtype=torch.long)  # Long 타입으로 변환
        # state를 원-핫 인코딩
        state_encoded = self.onehot_encoder.transform(np.array(state).reshape(-1, 1))[0]
        # cv2.imshow("img_path",img_path)
        # cv2.waitKey(0)
        # plt.imshow(img_path)
        # plt.show()
        # print("state", state)
        return img, state_encoded
# 훈련 데이터에 대해 OneHotEncoder 학습
onehot_encoder.fit(np.array(['G', 'L', 'R']).reshape(-1, 1))
# 훈련 및 테스트 데이터셋 생성
train_dataset = CustomDataset(images=images, states=states, transform=transform, train=True, onehot_encoder=onehot_encoder)
test_dataset = CustomDataset(images=images, states=states, transform=transform, train=False, onehot_encoder=onehot_encoder)
# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

# 간단한 CNN 모델 정의
class RoadClassifier(nn.Module):
    def __init__(self, num_classes=3):
        super(RoadClassifier, self).__init__()
        self.cnn = models.resnet18(pretrained=True)
        in_features = self.cnn.fc.in_features
        self.cnn.fc = nn.Linear(in_features, num_classes)
    def forward(self, x):
        return self.cnn(x)
# 모델, 손실 함수, 최적화기 정의
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RoadClassifier(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
# 학습하기
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        print('outputs', outputs[0], 'labels', labels[0])
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss / i}")
print("학습 완료")
# 테스트하기
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        predicted = torch.argmax(outputs.data, dim=1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()
print(f"Accuracy: {100 * correct / total}%")
torch.save(model.state_dict(), 'road_classifier_model.pth')
# loaded_model = RoadClassifier(num_classes=3).to(device)
# loaded_model.load_state_dict(torch.load('road_classifier_model.pth'))
# loaded_model.eval()

In [ ]:
img = cv2.imread('./line/_00014_R.png')
a = Image.fromarray(np.asarray(img)).convert('RGB')
# 이미지 변환 및 차원 추가
transformed_image = transform(a).unsqueeze(0).to(device)
result = model(transformed_image)
print(result)
plt.imshow(a)
plt.show()